In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from scipy import stats

sys.path.append('..')

from src.features.build_features import df, vocabulary, X, X_tfidf

In [2]:
df.head()

,Starttid,Sluttid,Navn,Rolle,Tekst,MeetingId,Parti,Regering,I_Regering
0,2009-10-06 12:04:01.713,2009-10-06 12:11:14.713,Lars Løkke Rasmussen,minister,Danmark er et godt land at leve i. Vi har bygg...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
1,2009-10-06 12:11:14.713,2009-10-06 12:14:49.713,Lars Løkke Rasmussen,minister,"Det er ikke den eneste mulighed, vi skal gribe...",2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
2,2009-10-06 12:14:49.713,2009-10-06 12:19:47.713,Lars Løkke Rasmussen,minister,For det første – og helt afgørende: Vi skal ho...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
3,2009-10-06 12:19:47.713,2009-10-06 12:25:11.713,Lars Løkke Rasmussen,minister,Velstand og vækst er i virkeligheden de to byg...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
4,2009-10-06 12:25:11.713,2009-10-06 12:29:01.713,Lars Løkke Rasmussen,minister,Regeringen arbejder på hele velfærdsområdet me...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True


In [10]:
feature_word = [word for word, count in sorted(vocabulary.items(), key = lambda kv:(kv[1], kv[0]))]

In [3]:
X

<160444x23574 sparse matrix of type '<class 'numpy.int64'>'
	with 9014135 stored elements in Compressed Sparse Row format>